In [1]:
# This gets rid of Jupyter's screen-realestate-wating-margin-of-gray.
#

from IPython.core.display import display, HTML, Markdown
display(HTML(
    '<style>'
        '#notebook { padding-top:10px !important; } ' 
        '.container { width:95% !important; } '
        '.end_space { min-height:5px !important; } '
    '</style>'
))

# Customize Values

In [2]:
# Adjust these directories to where you put your stuff
import os

commonsRoot = os.path.expanduser("~/IdeaProjects/commons")                           


# Set up Environments

In [3]:
#
# Include the Python codes in the recommendations project into Jupyter's libraries
#

import sys
from pathlib import Path

for d in [commonsRoot, ]:
    sourceDir = d + '/src/main'
    if sourceDir not in sys.path:
        sys.path.append(sourceDir)


In [4]:
import time

# Experiment with Transformer

In [5]:
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration

In [14]:

# mname = "facebook/blenderbot-400M-distill"
mname = "facebook/blenderbot-1B-distill"
model = BlenderbotForConditionalGeneration.from_pretrained(mname)
tokenizer = BlenderbotTokenizer.from_pretrained(mname)
UTTERANCE = "My friends are cool but they eat too many carbs."

statTime = time.time()
inputs = tokenizer([UTTERANCE], return_tensors="pt")
reply_ids = model.generate(**inputs)
print(tokenizer.batch_decode(reply_ids))
print(time.time() - statTime)

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.68G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.4k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

["<s> That's unfortunate. Are they trying to lose weight or are they just trying to be healthier?</s>"]
7.457543134689331


In [15]:
statTime = time.time()
UTTERANCE = "no, they just like sweets"
inputs = tokenizer([UTTERANCE], return_tensors="pt")
reply_ids = model.generate(**inputs)
print(tokenizer.batch_decode(reply_ids))
print(time.time() - statTime)

["<s> That's what I was thinking, but I don't know if I want to risk it.</s>"]
7.560004711151123


In [49]:
statTime = time.time()
# UTTERANCE = '''
# your persona: I like cheese
# your persona: I am from New York City
# Hi, where are you from
# i ' m from the city of new york . i love cheese . what do you like to eat ?
# do you like cheese?
# yes , i love it . it ' s one of my favorite foods . what ' s your favorite food ?
# do you know what is piave?
# '''
UTTERANCE = '''
you: I like cheese
you: I am from New York City
Hi, how are you?  Do you like Piave?
'''

inputs = tokenizer([UTTERANCE], return_tensors="pt", padding=True)
reply_ids = model.generate(**inputs)
print(tokenizer.batch_decode(reply_ids))
print(time.time() - statTime)

["<s> I love it! It's one of the most popular foods in the world. What do you like to eat?</s>"]
7.227740049362183


In [39]:
inputs

{'input_ids': tensor([[ 206,   96,  512,  800,   72,   33,  281,  398, 4686,  206,   96,  512,
          800,   72,   33,  281,  632,  482, 2310, 6210, 5203,  206,  206,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

# Experiment with ParlAI

### Requirements
```
pip install parlai
pip install subword-nmt
```

In [18]:
from parlai.core.agents import create_agent_from_model_file
# blender_agent = create_agent_from_model_file("zoo:blender/blender_400M/model")
blender_agent = create_agent_from_model_file("zoo:blender/blender_3B/model")

# forget everything. Important if you run this multiple times.
blender_agent.reset()

# concatenate the persona and the first thing the human says
first_turn = "\n".join([
    "your persona: I like cheese",
    "your persona: I am from New York City",
    "Hi, where are you from"
])
# Model actually witnesses the human's text
blender_agent.observe({'text': first_turn, 'episode_done': False})
print(f"You said: {first_turn}")

# model produces a response
startTime = time.time()
response = blender_agent.act()
print("BlenderBot replied: {}".format(response['text']))
print(time.time() - startTime)
print()

# now another turn
second_turn = "do you like cheese?"
print(f"You said: {second_turn}")

startTime = time.time()
blender_agent.observe({'text': second_turn, "episode_done": False})
response2 = blender_agent.act()
print("BlenderBot replied: {}".format(response2['text']))
print(time.time() - startTime)

print()
print('-' * 40)
print()
print("BlenderBot's history view:")
print(blender_agent.history.get_history_str())

14:04:56 | loading dictionary from /Users/cjwang/opt/anaconda3/lib/python3.7/site-packages/data/models/blender/blender_3B/model.dict
14:04:57 | num words = 8008
14:05:25 | Total parameters: 2,696,268,800 (2,695,613,440 trainable)
14:05:25 | Loading existing model params from /Users/cjwang/opt/anaconda3/lib/python3.7/site-packages/data/models/blender/blender_3B/model
You said: your persona: I like cheese
your persona: I am from New York City
Hi, where are you from
BlenderBot replied: I'm from the city of New York, the most populous city in the United States.
15.416275978088379

You said: do you like cheese?
BlenderBot replied: Yes, I love cheese.  It's one of my favorite foods.  What about you?
16.74525499343872

----------------------------------------

BlenderBot's history view:
your persona: I like cheese
your persona: I am from New York City
Hi, where are you from  I'm from the city of New York, the most populous city in the United States.  do you like cheese?  Yes, I love cheese.  

In [19]:
# more turns from CJ
# third_turn = "do you know what is piave?"
third_turn = "how long do you take to go to the Jersey City?"

startTime = time.time()
blender_agent.observe({'text': third_turn, "episode_done": False})
response3 = blender_agent.act()
print("BlenderBot replied: {}".format(response3['text']))
print(time.time() - startTime)

print("BlenderBot's history view:")
print(blender_agent.history.get_history_str())

BlenderBot replied: I've never been to New Jersey, but I've heard it's a nice place to visit.
33.50209093093872
BlenderBot's history view:
your persona: I like cheese
your persona: I am from New York City
Hi, where are you from  I'm from the city of New York, the most populous city in the United States.  do you like cheese?  Yes, I love cheese.  It's one of my favorite foods.  What about you?  how long do you take to go to the Jersey City?  I've never been to New Jersey, but I've heard it's a nice place to visit.


In [20]:
response3

{'id': 'TransformerGenerator',
 'episode_done': False,
 'text': "I've never been to New Jersey, but I've heard it's a nice place to visit.",
 'beam_texts': [("I've never been to New Jersey, but I've heard it's a nice place to visit.",
   -6.780792713165283),
  ("I've never been to New Jersey, but I hear it's nice.  I've been to Philadelphia, though.",
   -7.047243595123291),
  ("I've never been, but I'd love to go.  I hear it's the most densely populated city in New York State.",
   -7.1058454513549805),
  ("I've never been to New Jersey, but I hear it's nice.  I've been to Philadelphia, which is the second-largest city in Pennsylvania.",
   -7.14259672164917),
  ("I've never been to New Jersey, but I hear it's nice.  I've been to Philadelphia though.",
   -7.26740026473999),
  ("I've never been, but I'd love to go.  I hear it's the most densely populated state in the US.",
   -7.279247283935547),
  ("I've never been, but I'd love to go.  I hear it's the most densely populated city in 

# Use Conversational Pipeline

In [87]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Conversation, ConversationalPipeline
from flask import Flask
from flask import request, jsonify

modelName = "facebook/blenderbot-1B-distill"
tokenizer = AutoTokenizer.from_pretrained(modelName)
model = AutoModelForSeq2SeqLM.from_pretrained(modelName)
nlp = ConversationalPipeline(model=model, tokenizer=tokenizer)

conversation = Conversation()


In [141]:
def chat(text):
    conversation.add_user_input(text)
    result = nlp([conversation], do_sample=False, max_length=1000)
    
    print(str(result))
    print(f">>>> {list(result.iter_texts())[-1]}")
    *_, last = result.iter_texts()
    print(last)
#     for is_user, text in result.iter_texts():
#         who = "Me:" if is_user else "Bot:"
#         print(f"{who} {text}")


In [89]:
def reset():
    global conversation
    conversation = Conversation()

In [126]:
def persona(text):
    conversation.add_user_input('Hello')
    conversation.append_response(text)
        
    # Put the user's messages as "old message".
    conversation.mark_processed()


In [136]:
reset()
persona("I live in New York City.  I like cheese.  I jog everyday.")

In [142]:
startTime = time.time()
print(chat("Empire State Building is NOT a park.  Do you run up and down the stairs?"))
print(time.time() - startTime)

Trimmed input from conversation as it was longer than 128 tokens.


Conversation id: 9c749996-8d66-4778-a4d9-7f12d6502af8 
user >> Hello 
bot >> I live in New York City.  I like cheese.  I jog everyday. 
user >> Empire State Building is NOT a park.  Do you run up and down the stairs? 
bot >>  Yes, I do.  It is the most populous city in the United States. 
user >> Empire State Building is NOT a park.  Do you run up and down the stairs? 
bot >>  No, I don't.  The stairs are too steep for me to run up. 
user >> Empire State Building is NOT a park.  Do you run up and down the stairs? 
bot >>  No, but I do jog on the staircase.  That is the only way I can jog. 

>>>> (False, ' No, but I do jog on the staircase.  That is the only way I can jog.')
(False, ' No, but I do jog on the staircase.  That is the only way I can jog.')
None
9.312549114227295


In [81]:
conversation.add_user_input("I like NYC")
result = nlp([conversation], do_sample=False, max_length=1000)
dir(result)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_user_input',
 'append_response',
 'generated_responses',
 'iter_texts',
 'mark_processed',
 'new_user_input',
 'past_user_inputs',
 'uuid']